In [1]:
# importing necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [7]:
# creating empty container for storing jobs information
joblist = []

In [8]:
# creating the html parser
def extract(page):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}
    url = f'https://malaysia.indeed.com/jobs?q=python+developer&l=malaysia&start={page}'
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup

In [9]:
# transforming raw HTML content into structured format
def transform(soup):
    divs = soup.find_all('div', class_='job_seen_beacon')
    for item in divs:
        # Extract title
        title = item.find('h2').text.strip()
        
        # Extract company name
        company = item.find('span', class_='css-92r8pb eu4oa1w0').text.strip()

        # Extract salary
        salary = None
        salary_container = item.find('div', class_='jobMetaDataGroup css-pj786l eu4oa1w0')
        if salary_container:
            salary_divs = salary_container.find_all('div', class_='css-1cvvo1b eu4oa1w0')
            for div in salary_divs:
                # print(f"Found metadata div: {div.text}")
                if 'RM' in div.text:
                    salary = div.text
                    break
            
        # Extract summary
        summary = item.find('div', class_='css-9446fg eu4oa1w0').text.strip().replace('\n', ' ')
        
        job = {
            'title': title,
            'company': company,
            'salary': salary,
            'summary': summary
        }
        joblist.append(job)
    
    return joblist


In [10]:
# iterating through pages
for i in range(0,40,10):
    print(f'Getting page, {i}')
    c = extract(i)
    transform(c)

Getting page, 0
Getting page, 10
Getting page, 20
Getting page, 30


In [11]:
df = pd.DataFrame(joblist)
print(df.head(10))

                                               title  \
0              Full Stack Developer (Chat platforms)   
1  Full Stack Developer (ReactJS/NodeJS/C#/Python...   
2                           AI Developer - Full Time   
3                                Fullstack Developer   
4  Web Programmer (Web Developer) #PHP and Larave...   
5                               Python Web Developer   
6                                 Software Developer   
7                          Junior Software Developer   
8                                 Software Developer   
9                            Python Developer Intern   

                                   company                        salary  \
0                               JP Caliber             RM 10,000 a month   
1         LYJ Events & Marketing sdn. bhd.                          None   
2                       IMAGINE AI SDN BHD   RM 3,200 - RM 4,000 a month   
3                              RinggitPlus                          None   
4  

In [7]:
# saving the scrapped data into csv
df.to_csv('jobs.csv')